In [ ]:
import os

import pandas as pd
import mlflow
from dotenv import load_dotenv, find_dotenv
load_dotenv()
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, 
    SplineTransformer, 
    QuantileTransformer, 
    RobustScaler,
    PolynomialFeatures,
    KBinsDiscretizer,
)

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.xcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "AP2T"
RUN_NAME = "preprocessing" 
REGISTRY_MODEL_NAME = "test_01"

In [3]:
obj_df = df.select_dtypes(include="object")

In [ ]:
print(obj_df.head(2))

In [ ]:
# categorical features
cat_columns = ["type", "payment_method", "internet_service", "gender"]

# creating a OneHotEncoder object
encoder_oh = OneHotEncoder(categories='auto', handle_unknown='ignore', max_categories=10, sparse_output=False, drop='first')

# applying OneHotEncoder
encoded_features = encoder_oh.fit_transform(df[cat_columns])

# features transformation
encoded_df = encoder_oh.get_feature_names_out(cat_columns)
encoded_feature_names = encoder_oh.get_feature_names_out(cat_columns)
encoded_df = pd.DataFrame(encoded_features, columns=encoded_feature_names, index=df.index)

obj_df = pd.concat([obj_df, encoded_df], axis=1)

obj_df.head(2)

In [5]:
num_columns = ["monthly_charges", "total_charges"]

n_knots = 3
degree_spline = 4
n_quantiles=100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None

In [ ]:
print(df[num_columns].isna().sum())
print(df[num_columns].head(2))

In [6]:
df[num_columns] = df[num_columns].fillna(df[num_columns].mean())


In [ ]:
print(df[num_columns].isna().sum())
print(df[num_columns].head(2))

In [ ]:
num_df = df[num_columns]

# SplineTransformer
encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline)
encoded_features = encoder_spl.fit_transform(df[num_columns].to_numpy())
encoded_df = pd.DataFrame(
    encoded_features, 
    columns=encoder_spl.get_feature_names_out(num_columns)
)
num_df = pd.concat([num_df, encoded_df], axis=1)


# QuantileTransformer
encoder_q = QuantileTransformer(n_quantiles=n_quantiles)
encoded_features = encoder_q.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_q.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_q_{n_quantiles}" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


# RobustScaler
encoder_rb = RobustScaler()
encoded_features = encoder_rb.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_rb.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_robust" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)


# PolynomialFeatures
encoder_pol = PolynomialFeatures(degree=degree)
encoded_features = encoder_pol.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_pol.get_feature_names_out(num_columns))
encoded_df.drop(encoded_df.columns[:1 + len(num_columns)], axis=1, inplace=True)

# KBinsDiscretizer
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample)
encoded_features = encoder_kbd.fit_transform(df[num_columns].to_numpy())

encoded_df = pd.DataFrame(encoded_features, columns=encoder_kbd.get_feature_names_out(num_columns))
encoded_df.columns = [col + f"_bin" for col in num_columns]
num_df = num_df = pd.concat([num_df, encoded_df], axis=1)


num_df.head(2)

In [ ]:
numeric_transformer = ColumnTransformer(
    transformers=[('spl', encoder_spl, num_columns), 
                  ('q', encoder_q, num_columns), 
                  ('rb', encoder_rb, num_columns), 
                  ('pol', encoder_pol, num_columns), 
                  ('kbd', encoder_kbd, num_columns)])

categorical_transformer = Pipeline(steps=[('encoder', encoder_oh)])

preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, num_columns), 
('cat', categorical_transformer, cat_columns)], n_jobs=-1)

encoded_features = preprocessor.fit_transform(df)

transformed_df = pd.DataFrame(encoded_features, columns=preprocessor.get_feature_names_out())

df = pd.concat([df, transformed_df], axis=1)
df.head(2)

In [ ]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.xcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "AP2T"
RUN_NAME = "preprocessing" 
REGISTRY_MODEL_NAME = "test_01"

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    print(run_id)
    mlflow.sklearn.log_model(preprocessor, "column_transformer")

2025/03/07 16:42:42 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


da026a10c333400da167ea17798bd355


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from category_encoders import CatBoostEncoder

X_tr, X_val, y_tr, y_val = train_test_split(df, df['target'], stratify=df['target'])

cat_features = df.select_dtypes(include='object')
potential_binary_features = cat_features.nunique() == 2
binary_cat_features = cat_features[potential_binary_features[potential_binary_features].index]
other_cat_features = cat_features[potential_binary_features[~potential_binary_features].index]
num_features = df.select_dtypes(['float']) 
binary_cols = binary_cat_features.columns.tolist()
non_binary_cat_cols = other_cat_features.columns.tolist()
num_cols = num_features.columns.tolist()

preprocessor = ColumnTransformer(
    [
    ('binary', OneHotEncoder(drop='if_binary'), binary_cols),
    ('cat', CatBoostEncoder(), non_binary_cat_cols),
    ('num', StandardScaler(), num_cols)
    ],
    remainder='drop',
    verbose_feature_names_out=False
)
model = CatBoostClassifier(auto_class_weights='Balanced')

pipeline = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('model', model)
    ]
)

pipeline.fit(df, df['target'])

y_pred = pipeline.predict(X_val)
y_pred_proba = pipeline.predict_proba(X_val)[:, 1]

print('f1:', f1_score(y_val, y_pred))
print('roc_auc:', roc_auc_score(y_val, y_pred_proba))

In [ ]:
import os
import mlflow


EXPERIMENT_NAME = "outflow_fio"
RUN_NAME = "model0_reg"
REGISTRY_MODEL_NAME = "outflow_model_ap"


os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.xcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")

pip_requirements = "requirements.txt"
signature = mlflow.models.infer_signature(X_test, prediction)
input_example = X_test[:10]
metadata = {'model_type': 'monthly'}


experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    model_info = mlflow.catboost.log_model( 
        cb_model=model,
        await_registration_for=60,
        artifact_path="models",
        pip_requirements=pip_requirements,
        input_example=input_example,
        metadata=metadata,
        signature=signature,
        registered_model_name=REGISTRY_MODEL_NAME,		)